In [5]:
pip install pyspark


Note: you may need to restart the kernel to use updated packages.


In [1]:

pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [61]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, max, min, row_number, sum as spark_sum, month, date_format
from pyspark.sql.window import Window

# Initialize Spark Session
spark = SparkSession.builder.appName("CovidDataAnalysis").getOrCreate()



In [62]:
df = df.withColumn("Name of State / UT", lower(col("Name of State / UT")))
df.select('Name of State / UT').show(truncate=False)

+------------------+
|Name of State / UT|
+------------------+
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
+------------------+
only showing top 20 rows



In [63]:
#day with the greatest number of COVID cases:
max_cases_day = df.orderBy(col("New cases").desc()).select("Date").first()
print(f"The day with the greatest number of COVID cases: {max_cases_day['Date']}")


The day with the greatest number of COVID cases: 2020-07-26


In [82]:
#The state with the second-largest number of COVID cases

# Aggregate total confirmed cases by state
aggregated_df = df.groupBy("Name of State / UT") \
                  .agg(spark_sum("Total Confirmed cases").alias("Total Confirmed cases"))

# Create window spec and rank states
window_spec = Window.orderBy(col("Total Confirmed cases").desc())
ranked_states = aggregated_df.withColumn("rank", row_number().over(window_spec))

second_largest_state = top_two_states.filter(col("rank") == 2).first()

print(f"The state with the second-largest number of COVID cases: {second_largest_state['Name of State / UT']} with {second_largest_state['Total Confirmed cases']} cases")

The state with the second-largest number of COVID cases: tamil nadu with 7847083.0 cases


In [65]:
#The state with the least number of deaths
state_least_deaths = df.groupBy("Name of State / UT") \
                       .agg(spark_sum("Death").alias("Total_Deaths")) \
                       .orderBy(col("Total_Deaths").asc()) \
                       .select("Name of State / UT")
print(f"The state with the least number of deaths: {state_least_deaths.first()['Name of State / UT']}")


The state with the least number of deaths: union territory of ladakh


In [84]:
#The state with the lowest Death to Total Confirmed cases ratio
df = df.withColumn("Death_to_Confirmed_ratio", col("Death") / col("Total Confirmed cases"))
state_lowest_ratio = df.orderBy(col("Death_to_Confirmed_ratio").asc()).select("Name of State / UT").first()
print(f"The state with the lowest Death to Total Confirmed cases ratio: {state_lowest_ratio['Name of State / UT']}")


The state with the lowest Death to Total Confirmed cases ratio: puducherry


In [86]:
#The month with the most new recovered cases
df = df.withColumn("Month", month(col("Date")))
most_recovered_month = df.groupBy("Month").agg(spark_sum("New recovered").alias("Total New Recovered")).orderBy(col("Total New Recovered").desc()).select("Month").first()

month_name = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
}

print(f"The month with the most new recovered cases: {month_name[most_recovered_month['Month']]}")


The month with the most new recovered cases: July
